<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [1]:
!pip install bertopic[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

## Connect your Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [5]:
# The bibliometrics folder
ROOT_FOLDER_PATH = f"drive/MyDrive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q312_utokyo'

analysis_id = 'a01_tm__f01_e01__km01'

# Filtered label
settings_directive = "settings_analysis_directive_2024-11-02-15-36.json"

In [6]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [12]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [8]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [9]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [10]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [13]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}")
print(df.shape)
df.head()

(11120, 39)


,X_N,uuid,PT,AU,TI,SO,LA,DT,DE,ID,...,VL,IS,BP,EP,AR,DI,PG,WC,SC,UT
0,1,41e14258-945f-4ec9-9e0a-e58c87752c10,J,"Kishi, T; Kobayashi, K; Sasagawa, K; Sakimura,...",Automated analysis of a novel object recogniti...,BEHAVIOURAL BRAIN RESEARCH,English,Article,Animal behavior; Novel object recognition test...,ANXIETY; MODELS,...,476,NaN,NaN,NaN,115278,10.1016/j.bbr.2024.115278,5,Behavioral Sciences; Neurosciences,Behavioral Sciences; Neurosciences & Neurology,WOS:001333148000001
1,2,f0daa586-2560-43ef-8132-f3b87f9167a3,J,"Berenger, F; Tsuda, K",An ANI-2 enabled open-source protocol to estim...,JOURNAL OF COMPUTATIONAL CHEMISTRY,English,Article; Early Access,ANI-2x; docking; ligand; LIT-PCBA; MC; QM; SBV...,OPEN-SOURCE PACKAGE; FORCE-FIELD,...,NaN,NaN,NaN,NaN,NaN,10.1002/jcc.27478,11,"Chemistry, Multidisciplinary",Chemistry,WOS:001328968200001
2,3,23327f73-de46-43e9-8796-decc6950c06d,J,"Kato, S; Ono, S",Accelerated Approval for Cancer Drugs in the U...,JOURNAL OF PHARMACEUTICAL INNOVATION,English,Article,Accelerated approval; Oncology; Evidence and a...,END-POINTS; ONCOLOGY,...,19,5,NaN,NaN,54,10.1007/s12247-024-09851-9,12,Pharmacology & Pharmacy,Pharmacology & Pharmacy,WOS:001304011000001
3,4,dff88211-4a40-4566-a7c0-1856b531092f,J,"Kojima, K; Chambers, JK; Yoshizawa, M; Fujioka...",Pathological features of intrathoracic histioc...,JOURNAL OF VETERINARY MEDICAL SCIENCE,English,Article,histiocytic sarcoma; spiny rat; Tokudaia osime...,CHROMOSOME,...,86,10,1076,1080,NaN,10.1292/jvms.24-0185,5,Veterinary Sciences,Veterinary Sciences,WOS:001338099800007
4,5,5e23ca4b-52e5-47ad-9812-1e7a65f2b2a8,J,"Delgado-Munoz, J; Matsunaka, R; Hiraki, K",Classification of Known and Unknown Study Item...,BRAIN SCIENCES,English,Article,long term memory; familiarity; electroencephal...,ERP; RECOGNITION; FAMILIARITY; RECOLLECTION,...,14,9,NaN,NaN,860,10.3390/brainsci14090860,21,Neurosciences,Neurosciences & Neurology,WOS:001323906200001




---



## PART 2: Topic Model

In [14]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [15]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [16]:
# Combine embeddings
documents = corpus.text.to_list()

In [38]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [17]:
len(documents)

11120

In [18]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [19]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [21]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2024-11-02 06:41:31,593 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-02 06:42:16,351 - BERTopic - Dimensionality - Completed ✓
2024-11-02 06:42:16,354 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-02 06:42:17,078 - BERTopic - Cluster - Completed ✓
2024-11-02 06:42:17,098 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-02 06:42:45,870 - BERTopic - Representation - Completed ✓


In [22]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,0,279,0_magnetic_spin_field_hall,"[magnetic, spin, field, hall, magnetization, p...",[magnetic anisotropy cecosi high magnetic fiel...
1,1,261,1_galaxies_similar_galaxy_mass,"[galaxies, similar, galaxy, mass, stellar, sta...",[photon burst clears earliest dusty galaxies: ...
2,2,229,2_cells_cancer_cell_tumor,"[cells, cancer, cell, tumor, expression, immun...",[characterization kif20b novel prognostic biom...
3,3,214,3_theory_usepackagemathrsfs usepackageupgreek ...,"[theory, usepackagemathrsfs usepackageupgreek ...",[effective brane field theory higher-form symm...
4,4,200,4_patients_95_outcomes_ci,"[patients, 95, outcomes, ci, risk, heart, arte...",[clinical outcomes mitral valve surgery atrial...
...,...,...,...,...,...
107,107,38,107_patients_fgf23_hearing_hearing loss,"[patients, fgf23, hearing, hearing loss, pain,...",[case report: guanfacine methylphenidate impro...
108,108,35,108_psii_rhodopsins_chromophore_light,"[psii, rhodopsins, chromophore, light, rhodops...",[chromophore-protein interactions affecting po...
109,109,35,109_equations_problem_solutions_solution,"[equations, problem, solutions, solution, equa...",[characterization f-concavity preserved dirich...
110,110,34,110_asteroid_phobos_asteroids_dimorphos,"[asteroid, phobos, asteroids, dimorphos, bould...",[ultraviolet visible reflectance spectra phobo...


In [23]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [24]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

112

In [25]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [26]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('magnetic', 0.026596785516081942),
 ('spin', 0.01774999412542823),
 ('field', 0.008884241322670289),
 ('hall', 0.008818804380757694),
 ('magnetization', 0.008690627632162506),
 ('phase', 0.007211351594195054),
 ('effect', 0.007109006204478887),
 ('topological', 0.006577423864781797),
 ('magnetic field', 0.0063011913893084295),
 ('state', 0.006066045300763887)]

In [27]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['magnetic anisotropy cecosi high magnetic field investigated high magnetic field properties ce-based kondo lattice cecosi means magnetization, magnetocaloric effect, contactless resistivity, specific heat, magnetic torque measurements pulsed high magnetic fields 55 t. cecosi proposed unconventional ordered phase called phase ii, appears paramagnetic phase antiferromagnetic phase iii. magnetization, magnetocaloric effect, contactless resistivity measurements determine field-angle dependence phase ii-iii boundary. magnetic torque measurements detect fourfold symmetry magnetic anisotropy high magnetic fields phases, implying magnetic hard axes (100) (110) plane tetragonal structure. comparison mean-field calculation magnetization data, determine crystal electric field energy scheme, agrees previous inelastic neutron scattering experiment. successfully reproduce fourfold magnetic anisotropy calculation, attributed hybridization crystal electric field wave functions. hybridization increase

In [ ]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [28]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': KMeans(n_clusters=112),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 30,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [29]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [30]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 12/12 [00:39<00:00,  3.27s/it]


In [36]:
topic_distr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [31]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

,UT,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document,Score,X_C,level0,cl99,cl-99
0,WOS:001333148000001,automated analysis novel object recognition te...,59,59_speech_language_gaze_participants,"[speech, language, gaze, participants, speaker...",[developmental pattern native non-native speec...,speech - language - gaze - participants - spea...,False,0.000000,59,59,False,False
1,WOS:001328968200001,ani-2 enabled open-source protocol estimate li...,41,41_protein_receptor_peptide_binding,"[protein, receptor, peptide, binding, proteins...",[structural dynamic insights activation μ-opio...,protein - receptor - peptide - binding - prote...,False,0.134972,41,41,False,False
2,WOS:001304011000001,accelerated approval cancer drugs united state...,13,13_patients_cancer_chemotherapy_survival,"[patients, cancer, chemotherapy, survival, col...",[adjuvant nivolumab plus chemotherapy versus p...,patients - cancer - chemotherapy - survival - ...,False,0.000000,13,13,False,False
3,WOS:001338099800007,pathological features intrathoracic histiocyti...,51,51_cancer_patients_mutations_genomic,"[cancer, patients, mutations, genomic, cases, ...","[genomic landscape endometrial, ovarian, cervi...",cancer - patients - mutations - genomic - case...,False,0.000000,51,51,False,False
4,WOS:001323906200001,classification known unknown study items memor...,18,18_motor_cortex_task_auditory,"[motor, cortex, task, auditory, walking, movem...",[stable sound decoding despite modulated sound...,motor - cortex - task - auditory - walking - m...,False,0.000000,18,18,False,False


In [32]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [33]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2024-11-02 06:45:14,320 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

